# Famotidine Covid

This is a test case to see whether Answer Merging helps with the arbitrary query problem. The problem is: I want to know the most interesting way in which these two entities are related.  What is it?  One option is to do question rewriting.  But another (this one) is to general queries and then work on merging them.

In [1]:
import requests
import json
import time
import pandas as pd

#GammaViewer available from https://github.com/ranking-agent/gamma-viewer
from GammaViewer import GammaViewer
from IPython.display import display

In [2]:
def getlabel(curie):
    url = 'https://nodenormalization-sri.renci.org/get_normalized_nodes'
    try:
        r = requests.get(url,params={'curie':curie}).json()
        return r[curie]['id']['label']
    except:
        return ''

class RobokopMessenger:
    def __init__(self):
        self.url = 'http://stars-k6.edc.renci.org:32140'
    def pipeline(self,request,full = True):
        #normalize question
        response = requests.post( f'{self.url}/normalize', json=request )
        normalized = response.json()
        
        #answer question
        request = { 'message': normalized, }
        response = requests.post( f'{self.url}/answer', json=request )
        answered = response.json()
        if not full:
            return answered
        
        #Yank
        request = { 'message': answered, }
        response = requests.post( f'{self.url}/yank', json=request )
        filled = response.json()
        
        #support
        request = { 'message': filled, }
        response = requests.post( 'http://robokop.renci.org:4868/support', json=request )
        supported = response.json()
        #weight
        request = { 'message': supported, }
        response = requests.post( 'http://robokop.renci.org:4868/weight_correctness', json=request )
        weighted = response.json()
        #score
        request = { 'message': weighted, }
        response = requests.post( 'http://robokop.renci.org:4868/score', json=request )
        scored = response.json()
        return scored

    
robokop=RobokopMessenger()

In [3]:
def create_message():
    query_graph = {
    "nodes": [
        {
            #Covid-19
            "id": "a",
            "type": "disease",
            "curie": "MONDO:0100096"
        },
        {
            "id": "b",
            "type": "named_thing"
        },
        {
            #Famotidine
            "id": "c",
            "type": "chemical_substance",
            "curie": "CHEBI:4975"
        }
    ],
    "edges": [
        {
            "id": "ab",
            "source_id": "a",
            "target_id": "b"
        },
        {
            "id": "bc",
            "source_id": "c",
            "target_id": "b"
        }
    ]
    }
    message = {"message": {"query_graph": query_graph,
                       'knowledge_graph':{'url': 'bolt://stars-k6.edc.renci.org:31333',
            'credentials': {
                'username': 'neo4j',
                'password': 'ncatsgamma',
            },}, 
                           'results':[{"node_bindings": [],"edge_bindings": []}]}}
    return message

In [4]:
message = create_message()
react_component = GammaViewer(props={"data":message['message']})
display(react_component)

In [5]:
robokop_result = robokop.pipeline(message,full=True)
print( json.dumps(robokop_result,indent=2))

{
  "knowledge_graph": {
    "edges": [
      {
        "ctime": [
          1592973258.6295419
        ],
        "edge_source": [
          "cord19_scigraph_v7"
        ],
        "enrichment_p": 7.200162544618264e-26,
        "id": "6495101038d3ebd2aef30675f0bef227",
        "num_publications": 0.8943779167033549,
        "predicate_id": "owl:ObjectProperty",
        "publications": [],
        "relation": [
          "SEMMEDDB:ASSOCIATED_WITH"
        ],
        "relation_label": [
          "related_to"
        ],
        "source_database": [
          "cord19_scigraph_v7"
        ],
        "source_id": "CHEBI:29320",
        "target_id": "CHEBI:4975",
        "type": "related_to",
        "weight": 1
      },
      {
        "count": 3,
        "edge_source": "https://www.drugbank.ca/covid-19",
        "id": "4c4115b44968337424dd20dba4f70e0c",
        "predicate_id": "ROBOKOVID:in_clinical_trial_for",
        "relation": "in_clinical_trial_for",
        "relation_label": "in_cli

In [9]:
with open('famcov.json','w') as outf:
    json.dump(robokop_result,outf,indent=2)

## Coalesce Answers by Property

In [7]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'property'}
p_r = requests.post(url,json=robokop_result,params=params)
print(p_r.status_code)
property_result = p_r.json()

200


In [8]:
property_result['results']

[]

In [9]:
results = property_result['results']
num_grouped=[]
p_values=[]
properties=[]
for result in results:
    chemnode = result['node_bindings'][2]
    for p,prop in zip(chemnode['p_values'],chemnode['properties']):
        num_grouped.append(len(chemnode['kg_id']))
        p_values.append(p)
        properties.append(prop)
df = pd.DataFrame( {'NumberGrouped': num_grouped, 'p_value':p_values, 'PropertyGroup':properties})
df.sort_values(by='p_value',inplace=True)
df

,NumberGrouped,p_value,PropertyGroup


## Coalesce Answers by Ontology

In [6]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'ontology'}
o_r = requests.post(url,json=robokop_result,params=params)
print(o_r.status_code)
ontology_result = o_r.json()
print( len(ontology_result['results']))

200
18


In [7]:
results = ontology_result['results']
for result in results:
    print('\n================')
    chemnode = result['node_bindings'][1]
    sclass = chemnode['superclass']
    label = getlabel(sclass)
    nodes = chemnode['kg_id']
    labels = {x: f'{getlabel(x)}' for x in nodes }
    for x in nodes:
        print(f'{labels[x]} ({x})')
    print('----have superclass----')
    print(f'{label} ({sclass})')


respiratory failure (MONDO:0021113)
acute respiratory distress syndrome (MONDO:0006502)
----have superclass----
respiratory failure (MONDO:0021113)

dexamethasone (CHEBI:41879)
prednisone (CHEBI:8382)
cortisol (CHEBI:17650)
----have superclass----
glucocorticoid (CHEBI:24261)

methylprednisolone succinate (CHEBI:135765)
dexamethasone (CHEBI:41879)
prednisone (CHEBI:8382)
cortisol (CHEBI:17650)
----have superclass----
corticosteroid hormone (CHEBI:36699)

respiratory failure (MONDO:0021113)
acute respiratory distress syndrome (MONDO:0006502)
----have superclass----
respiratory failure (MONDO:0021113)

dexamethasone (CHEBI:41879)
prednisone (CHEBI:8382)
cortisol (CHEBI:17650)
----have superclass----
glucocorticoid (CHEBI:24261)

methylprednisolone succinate (CHEBI:135765)
dexamethasone (CHEBI:41879)
prednisone (CHEBI:8382)
cortisol (CHEBI:17650)
----have superclass----
corticosteroid hormone (CHEBI:36699)

respiratory failure (MONDO:0021113)
acute respiratory distress syndrome (MONDO:00

In [18]:
from copy import deepcopy
onrd = deepcopy(ontology_result)
for n in onrd['query_graph']['nodes']:
    if 'curie' in n and n['curie'] is None:
        del[n['curie']]
for n in onrd['knowledge_graph']['nodes']:
    if 'name' not in n:
        label = getlabel(n['id'])
        n['name'] = label
for e in onrd['knowledge_graph']['edges']:
    e['id'] = str(e['id'])
for r in onrd['results']:
    for eb in r['edge_bindings']:
        eb['kg_id'] = [ str(x) for x in eb['kg_id']]
react_component = GammaViewer(props={"data":onrd})
display(react_component)

## Coalescence by Graph

Coalescence by graph is similar to coalescence by ontology, but the new relationships can be more general than `is_a`.  

In [ ]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'graph'}
g_r = requests.post(url,json=robokop_result,params=params)
print(g_r.status_code)
graph_result = g_r.json()
print( len(graph_result['results']))

In [19]:
from copy import deepcopy
gd = deepcopy(graph_result)
for n in gd['query_graph']['nodes']:
    if 'curie' in n and n['curie'] is None:
        del[n['curie']]
for n in gd['knowledge_graph']['nodes']:
    if 'name' not in n:
        label = getlabel(n['id'])
        n['name'] = label
for e in gd['knowledge_graph']['edges']:
    e['id'] = str(e['id'])
for r in gd['results']:
    for eb in r['edge_bindings']:
        eb['kg_id'] = [ str(x) for x in eb['kg_id']]
react_component = GammaViewer(props={"data":gd})
display(react_component)

In [24]:
def draw_one(results,rnum):
    g = deepcopy(results)
    g['results']=[g['results'][rnum]]
    kgs = set()
    for nb in g['results'][0]['node_bindings']:
        for kgid in nb['kg_id']:
            kgs.add(kgid)
    kg = g['knowledge_graph']
    kg['nodes'] = [n for n in kg['nodes'] if n['id'] in kgs ]
    kg['edges'] = [e for e in kg['edges'] if (e['source_id'] in kgs) and (e['target_id'] in kgs)]
    react_component = GammaViewer(props={"data":g})
    display(react_component)

In [29]:
draw_one(gd,3)